In [1]:
import sys


no_of_cpu = 8
max_cores = 8
executor_mem = '56g'



Job_Name = 'Nishant: Policy_retail'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F


conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = ''
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx8g -Xms8g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
18712,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Read the table and load as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='policy_retail')\
            .filter(col('source')=='policy_retail').filter(col('status')=='Success').filter(col('records')!='0')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]
 

        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))2014-11-07 (2019, 12, 31 

    except Exception as E: 
        Max_last_run = datetime.datetime(2023, 12,1, 0, 0)  # 2014-11-07 this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                

        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark

def update_policy(conn_to,table_name,prod_gpdb_spark_options, schema="customermart"):
    
    conn_to.rollback()
#     gpdb_spark_options = prod_gpdb_spark_options


    primary_key = ['source_system_name','source_system_customer_id','policy_number','certificate_no','pol_incept_date','pol_exp_date']
    
    # these columns will be updated in case of duplicate records
    update_fields = ['reference_num','sum_insured','premiumamount','total_premium','product_name','channel','channel_new_2','lob','record_type_desc','record_type_cd','report_accounting_period','commit_timestamp','renl_cert_no','times_renewed_cnt','product_cd','product_cd_new','producer_cd','producername','producer_type','policy_counter','veh_model','veh_age','veh_make','veh_body','veh_engine_no','veh_chassis_no','no_claim_bonus','policy_type','fuel_type','uw_discount','uw_loading','business_type','txt_business_type','no_of_lives','veh_no','segment_type','policy_variant','mfg_year','ncb_percent','discount','major_line_cd','minor_line_cd','main_lob','pol_status_cd','group_category','plan_type','break_in_insurance_flag','break_in_insurance_days','exshowroomprice','irda_product_uin','txt_active_flag','status','txt_intermediary_cd','txt_status','accidentaldeathrider','rider_applicable','coverage_opted','age_building','personal_accident','public_lia','trip_type','trv_limit','purpose_of_trip','dep_date','return_date','travel_duration','destination_country','trip','lta_zone','information280','ped_declared_indicator','portability_required','deductible_amount','cumulative_bonus_earned','orphan_policy_indicator','orphan_policy_date','policy_term','ncb_flag','portability_indicator','renewed_in_grace_period']
    
    out_columns=['reference_num','source_system_name','source_system_customer_id','policy_number','certificate_no',
                   'pol_incept_date','pol_exp_date','sum_insured','premiumamount','total_premium','product_name','channel','channel_new_2','lob','record_type_desc','record_type_cd','report_accounting_period','commit_timestamp','renl_cert_no','times_renewed_cnt','product_cd','product_cd_new','producer_cd','producername','producer_type','policy_counter','veh_model','veh_age','veh_make','veh_body','veh_engine_no','veh_chassis_no','no_claim_bonus','policy_type','fuel_type','uw_discount','uw_loading','business_type','txt_business_type','no_of_lives','veh_no','segment_type','policy_variant','mfg_year','ncb_percent','discount','major_line_cd','minor_line_cd','main_lob','pol_status_cd','group_category','plan_type','break_in_insurance_flag','break_in_insurance_days','exshowroomprice','irda_product_uin','txt_active_flag','min_pol_incept_date','status','txt_intermediary_cd','txt_status','accidentaldeathrider','rider_applicable','coverage_opted','age_building','personal_accident','public_lia','trip_type','trv_limit','purpose_of_trip','dep_date','return_date','travel_duration','destination_country','trip','lta_zone','information280','ped_declared_indicator','portability_required','deductible_amount','cumulative_bonus_earned','orphan_policy_indicator','orphan_policy_date','policy_term','ncb_flag','portability_indicator','renewed_in_grace_period']
    # loading data from temp table to get column names    
#     df_1 = sqlContext.read.format("greenplum").options(**prod_gpdb_spark_options).option("dbschema",schema)\
#     .option("dbtable","{}_staging".format(table_name)).load()
#     out_columns = df_1.columns
#     out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    # this query insert records from temp table to target table when all the records in temp table are new and not present in target    
    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.\
    {table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
    # this query will be executed to update the records that are already present in target, new values will be assigned from temp table    
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
    # this query will be executed after update query to insert the remaining records    
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# # Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# # The function would first try to insert the records in original table. It that fails then it will upsert the records
# # Since there is no direct upsert query an update and insert query with where clause is used
# # This funciton just runs the SQL queries in GPDB and does not use spark

# def update_policy(conn_to,table_name,prod_gpdb_spark_options, schema="customermart"):
    
#     conn_to.rollback()
# #     gpdb_spark_options = prod_gpdb_spark_options


#     primary_key = ['source_system_name','source_system_customer_id','policy_number','certificate_no','policy_start_date',
#                    'policy_end_date']
    
#     # these columns will be updated in case of duplicate records
#     update_fields = ['reference_num','base_sum_insured','premiumamount','total_premium','irda_product_uin','intermediary_code','deductible_amount',
#                         'cumulative_bonus_earned','accidentaldeathrider','plan_type','lob','orphan_policy_indicator','orphan_policy_date',
#                         'break_in_insurance_flag','break_in_insurance_days','ped_declared_indicator','vehicle_registration_no','commit_timestamp',
#                         'policy_term','ncb_flag','portability_indicator','renewed_in_grace_period','product_name',
#                         'policy_type','channel','veh_model','veh_age','veh_make','veh_body','veh_engine_no','veh_chassis_no','fuel_type','uw_discount',
#                         'uw_loading','policy_renewal_count','record_type_desc','record_type_cd','product_cd','renl_cert_no','times_renewed_cnt',
#                         'no_of_lives','rider_applicable','coverage_opted','age_building','personal_accident','public_lia','trip_type','trv_limit',
#                         'purpose_of_trip','dep_date','return_date','travel_duration','destination_country','trip','lta_zone','segment_type',
#                         'report_accounting_period','producername','exshowroomprice','binddate','no_claim_bonus','policy_variant','mfg_year',
#                         'min_pol_incept_date','business_type','txt_business_type','ncb_percent','discount','producer_cd']
    
#     # loading data from temp table to get column names    
#     df_1 = sqlContext.read.format("greenplum").options(**prod_gpdb_spark_options).option("dbschema",schema)\
#     .option("dbtable","{}_staging".format(table_name)).load()
#     out_columns = df_1.columns
#     out_columns = [col for col in out_columns if "json" not in col]

#     cur_to = conn_to.cursor()  

#     # this query insert records from temp table to target table when all the records in temp table are new and not present in target    
#     only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.\
#     {table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
#     # this query will be executed to update the records that are already present in target, new values will be assigned from temp table    
#     update_query = """UPDATE {schema}.{table_name} orig
#                           SET
#                             {update_fields}
#                           FROM
#                             {schema}.{table_name}_staging temp
#                            WHERE 
#                             {primary_key}
#                        """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
#     # this query will be executed after update query to insert the remaining records    
#     insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
#                            SELECT {out_columns}
#                            FROM
#                              {schema}.{table_name}_staging temp
#                            WHERE
#                              NOT EXISTS (
#                              SELECT 1 FROM {schema}.{table_name} orig WHERE 
#                             {primary_key}
#                             )
#                         """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

#     try:
#         print("Inside try segment")
#         cur_to.execute(only_insert_query)
#         print("Executed Only insert query")
# #         update_progress(table_name,source,time_filter,records)
#     except Exception as e:
#         print("Inside except segment")
#         conn_to.rollback()
#         cur_to.execute(update_query)
#         print("Executed update query")
#         cur_to.execute(insert_query)
#         print("Executed insert query")
# #         update_progress(table_name,source,time_filter,records)
#     conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")


delta = d2 - d1
day=delta.days-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'pytz' is not defined
Traceback (most recent call last):
NameError: name 'pytz' is not defined



In [9]:
l=['B&M RT PD/BI','COMPREHENSIVEAVIATIONINSURANCE','MEGA RISK','TRADE CREDIT INSURANCE.','BUSINESSGUARD COMMERCIAL-MMKT',
                 'FINANCIALINSTITUTION','PROFESSIONAL INDEMNITY','AIG COMBINED','MONEYINS','COMMERCIALGENERALLIABILITYPRODUCT','ENERGY OTHER - CHEMICAL',
                 'MARINE CARGO E-MAR EXPORTS SMP','CYBER RISK PROTECTOR','CHEMICAL RT PD/BI','IAR PROPERTY MAJOR','CRISISMANAGEMENT','CAR/EAR','PUBLICLIABILITYACTS',
                 'BUSINESSGUARD ENERGY-CHEMICAL','WORKMENSCOMPENSATION','CASUALTY MAJOR','FIDELITYGUARANTEEINS','FIPI','PUBLICLIABILITYPRODUCT',
                 'CONSEQUENTIALLOSSINSURANCE','CLINICALLIABILITYINSURANCE','OIL & PETRO RT PD/BI','CRISISCOVERAGEINSURANCEPOLICY',
                 'ENERGY OTHER - UTILITIES','MARINEFREIGHTFORWARDLIABILITY','ENERGY - BICC','DIRECTORANDOFFICERLIABILITY','BUSINESSGUARDCOMMERCIAL',
                 'MANAGEMENTLIABILITYINSURANCEPOLICY','AIRPORT OPERATORS LIABILITY IN','CRISIS MANAGEMENT MAJOR','STD MARINE CROP CARGO RT MAJOR',
                 'INDUSTRIALALLRISKINSURANCE','MGMT LIABILITY MAJOR','CYBERRISKPROTECTOR','PROPERTY OTHER MAJOR','MARINE CARGO E-MAR DOMESTC SMP',
                 'MARINESPECIFICPOLICY','COMMERICALGENERALLIABILITY','PORTPACKAGEPOLICY','BUSINESSGUARD OFFICE - MMKRT','MANAGEMENT LIABILITY',
                 'ALLRISKSIGNS','CONTRACTORS PLANT & MACHINERY','UTILITIES RT PD/BI','MARINETEACROPPOLICY','BOILERPRESSUREPLANT',
                 'COMPREHENSIVEPRODUCTLIABILITY','BUSINESS GUARD ENERGY - O&P','TRADECREDITINSURANCE','MARINEADVANCEDLOSSOFPROFIT',
                 'ELECTRONIC EQUIPMENT INSURANCE','PUBLICLIABILITYINS','OPENPOLICYMARINECARGO','PROPERTY EEI','HIGHLIGHT - D & O','CRIMEMANAGER',
                 'ALLRISKPORTABLEINSURANCE','PROPERTY OTHER MMRKT','ELECTRONICEQUIPMENTINS','BAGGAGEINSU','PROPERTY MMRKT','PROPERTY BOILER-PRESSUREPLANT',
                 'ENVIRONMENTAL IMPAIRMENT LIABI','POLITICALRISK','BURGLARYINS','PROJECTINSURANCE','AIGCOMBINEDPACKAGEPOLICY','CPMINSURANCE',
                 'MACHINERYLOSSOFPROFITINS','PROPERTY B & M','BUSINESSGUARDAGRIPUMPSET','BUSINESSGUARDSOOKSHMAUDYAMSURAKSHA',
                 'BOILER & PRESSURE PLANT EQUIP','ENERGY MISCELLANEOUS','BUSINESSGUARD OFFICE - MAJOR','STD MARINE CARGO RT MMRKT',
                 'PLATEGLASSINS','INLAND MARINE CARGO RT MAJOR','BUSINESSGUARDLAGHUUDYAMSURAKSHA','POLLUTIONLEGALLIABILITY',
                 'ENERGYMISCELLANEOUS','STANDARDFIRESPECIALPERILINSURANCE','FIDELITY','MBDPOLICY','MARINE CARGO E-MAR IMPORTS SMP',
                 'MEGAPOLICY','PROPERTY MAJOR','PUBLIC LIABILITY ACT','EMPLOYEECOMPENSATION','AVIATION INSURANCE','BHARATGRIHARAKSHAPE',
                 'RPASINSURANCE','STD MARINE CARGO RT MAJOR',"WORKMEN'S COMPENSATION",'PROFESSIONALINDEMNITYPOLICY','BHARATGRIHARAKSHAAIGC',
                 'MARINE CARGO SPECPOL DOMESTIC','BUSINESSGUARD COMMERCIAL-MAJOR','MARINE CARGO SPECPOL IMPORTS','MARINE CARGO SPECPOL EXPORTS',
                 'MULTINATIONALPROGRAM','BUSINESSGUARDLAGHUUDYAMSURAKSHAPE']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.window import Window


col_str=["policy_no as policy_number ","pol_incept_date",'pol_exp_date','policy_variant','mfg_year',
         'ins_name_client_no as source_system_customer_id',
         'certificate_no' ,'proposal_no as reference_num ',"product_name",' business_type','txt_business_type','channel','record_type_desc','record_type_cd','producername',
         "product_cd","product_cd_new",'renl_cert_no','times_renewed_cnt','no_of_lives','segment_type','report_accounting_period',
        'veh_model','veh_age','veh_make','veh_body','veh_engine_no','veh_no','binddate','no_claim_bonus',
         'veh_chassis_no','binddate','policy_type','fuel_type','uw_discount','uw_loading','policy_counter','timestamp',
         'source_system','ncb_percent','discount','producer_cd','major_line_cd','minor_line_cd','main_lob','pol_status_cd',
'productgroup','lob','minor_line','channel_new_2','lob_new','txt_lob','sub_channel_new','new_sub_channel',
         'group_category','producer_type']
sales_all_details_df=load_gpdb_jdbc(col_str,'sales_all_details',"datamarts","binddate",time_filter)\
.filter(~(col('product_name').isin(l)))


In [12]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": user_prod,
                    "password": pwd_prod,
                    "dbschema": "customermart",
                    "dbtable": "policy_left_v2",#policy_retail_staging_2018_03,
                    "server.port": '1150-1170'}
sales_all_details_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()


# .filter(~(col('product_name').isin(l)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.sql.window import Window

w = Window.partitionBy(['policy_number','certificate_no','pol_incept_date','pol_exp_date','source_system']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
sales_all_details_df = sales_all_details_df.withColumn('row',F.row_number().over(w))
sales_all_details_df = sales_all_details_df.filter(F.col('row')==1).drop(sales_all_details_df.row).drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
sales_all_details_df.persist()
sales_all_details_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

438754

In [ ]:
# sales_all_details_df=sales_all_details_df.select( 'partner_appl_no', 'branch', 'report_accounting_period', 'receipt_no',
#                             'discount', 'endorsement_no', 'leader_name', 'group_category', 'main_lob', 'major_line_cd',
#                             'mfg_year', 'minor_line_cd', 'veh_make', 'veh_model', 'ri_inward_flag', 'zone',
#                             'product_cd_new', 'booking_date', 'policy_no', 'channel', 'certificate_no', 'class_peril_cd', 
#                             'policy_term', 'policy_type', 'policy_variant', 'coinsurance_flag', 'pol_incept_date', 
#                             'application_no', 'renl_cert_no', 'pol_exp_date', 'record_type_desc', 'record_type_cd', 'producer_cd',
#                             'producername', 'product_name', 'CommissionPercent', 'main_receipt_no', 'share', 'business_type', 
#                             'proposal_no', 'segment_type', 'sum_insured', 'rural_fg', 'branch_off_cd', 'producer_type', 'pin', 
#                             'prdr_branch_sub_cd', 'times_renewed_cnt', 'txt_business_type', 'covernotenumber', 'watts_policy_no',
#                             'coinsurance_cd', 'source_system', 'timestamp', 'veh_engine_no', 'veh_no', 'veh_body', 
#                             'source_system_customer_id', 'customer_id', 'clientname', 'client_city_name', 'endorsement_eff_date', 
#                             'prem_eff_date', 'sme_flag', 'fac_prem', 'ceded_vqst_prem', 'ceded_surplus_treaty_prem',
#                             'obligatory_prem', 'obligatory_comm', 'pol_office_cd', 'producer_name', 'ncb_percent',
#                             'pol_status_cd', 'product_cd', 'veh_age', 'txt_chasis_no', 'uw_discount', 
#                             'uw_loading', 'fuel_type', 'display_product_cd', 'policy_counter',
#                             'no_of_lives', 'row_num', 'product_eff_date', 'BindDate')

In [ ]:
# #extra
# sales_all_details_df=sales_all_details_df.withColumnRenamed('policy_no','policy_number').withColumnRenamed('proposal_no','reference_num')

In [15]:
sales_all_details_df=sales_all_details_df.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
grp_cols = ["policy_number", "certificate_no"]

sd_min=sales_all_details_df.groupBy(grp_cols).agg(min("pol_incept_date").alias("min_pol_incept_date"))

sd_min=sd_min.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

sales_all_details_df= sales_all_details_df.join(sd_min,grp_cols,"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.sql.window import Window

w = Window.partitionBy(['policy_number','certificate_no','pol_incept_date','pol_exp_date','source_system']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
sales_all_details_df = sales_all_details_df.withColumn('row',F.row_number().over(w))
sales_all_details_df = sales_all_details_df.filter(F.col('row')==1).drop(sales_all_details_df.row).drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": user_prod,
                    "password": pwd_prod,
                    "dbschema": "datamarts",
                    "dbtable": "sales_all_details",#policy_retail_staging_2018_03
                    "partitionColumn":"row_num",
                    "partitions": 8,
                    "server.port": '1150-1170'}
sales_all_details_df1 = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr('proposal_no as reference_num ',"policy_no as policy_number" ,'certificate_no', "pol_exp_date" ,
                               "pol_incept_date","binddate",'premiumamount','total_premium', 'sum_insured','source_system')\
.filter(~(col('product_name').isin(l)))

sales_all_details_df1=sales_all_details_df1.repartition("policy_number", "certificate_no",'pol_incept_date','pol_exp_date','source_system')

group_cols = ["policy_number", "certificate_no",'pol_incept_date','pol_exp_date','source_system']

sales_all_details_grp=sales_all_details_df1.groupBy(group_cols).agg(sum("premiumamount").alias("premiumamount"),sum("total_premium").alias("total_premium"),\
                              max('sum_insured').alias("base_sum_insured"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sales_all_details_grp=sales_all_details_grp.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
part_cols=["policy_number", "certificate_no",'pol_incept_date','pol_exp_date','source_system']

sales_all_details_df=sales_all_details_df.repartition(8,*part_cols)
sales_all_details_grp=sales_all_details_grp.repartition(8,*part_cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
sales_all_details_df= sales_all_details_df.join(sales_all_details_grp,group_cols,"inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# sales_all_details_df1=sales_all_details_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# sales_all_details_df.persist()
# sales_all_details_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# sales_all_details_df.select('policy_number').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
#                     "user": user_prod,
#                     "password": pwd_prod,
#                     "dbschema": "mappers",
#                     "dbtable": "direct_cons_comm_segment",
#                     "server.port": '1150-1170'}

# direct_cons_comm_segment_main = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# max_date= direct_cons_comm_segment_main.agg({"mnth_yr": "max"}).collect()[0]
# direct_cons_comm_segment_main = direct_cons_comm_segment_main.filter(col("mnth_yr")==max_date["max(mnth_yr)"])

# direct_cons_comm_segment = direct_cons_comm_segment_main.selectExpr('Minor_Line','Product_Name as Product_Name_mapper'\
#                                                                     ,'Min_Line_3','New_Direct','health_type')


# # LOB FROM direct_cons_comm_segment JOIN ON product_name

# from pyspark.sql.functions import upper, col, trim
# direct_cons_comm_segment = direct_cons_comm_segment.withColumn('Product_Name_mapper', trim(upper(col('Product_Name_mapper'))))
# sales_all_details_df = sales_all_details_df.withColumn('product_name', trim(upper(col('product_name'))))


# ########NEW LOGIC - FOR SUB_TOTAL_1 FIX (TEST)#################
# sales_all_details_df = sales_all_details_df.withColumn('product_name_substr',substring(trim(col('product_name')),1,45))

# condition = [sales_all_details_df.product_name_substr==direct_cons_comm_segment.Product_Name_mapper]

# sales_all_details_df = sales_all_details_df.join(direct_cons_comm_segment.select('Product_Name_mapper',col('Minor_Line').alias('lob')),\
#                                   condition,\
#                                   "left_outer")
# sales_all_details_df = sales_all_details_df.drop('Product_Name_mapper','product_name_substr')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
#                     "user": user_prod,
#                     "password": pwd_prod,
#                     "dbschema": "mappers",
#                     "dbtable": "channel_mapping_1",
#                     "server.port": '1150-1170'}

# channel_map=sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
#                         .selectExpr('Channel as Channel_temp','Channel_New_2','mnth_yr')
    
# max_date= channel_map.agg({"mnth_yr": "max"}).collect()[0]
# channel_map = channel_map.filter(col("mnth_yr")==max_date["max(mnth_yr)"]).drop("mnth_yr")

# condition_1 = [lower(trim(sales_all_details_df.channel)) == lower(trim(channel_map.Channel_temp))]

# sales_all_details_df = sales_all_details_df.join(channel_map,condition_1,"left_outer")\
#                      .drop(channel_map.Channel_temp)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from pyspark.sql.window import Window

w = Window.partitionBy(['policy_number','certificate_no','pol_incept_date','pol_exp_date','source_system']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
sales_all_details_df = sales_all_details_df.withColumn('row',F.row_number().over(w))
sales_all_details_df = sales_all_details_df.filter(F.col('row')==1).drop(sales_all_details_df.row).drop_duplicates()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
sales_all_details_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system', 'policy_variant', 'mfg_year', 'source_system_customer_id', 'reference_num', 'product_name', 'business_type', 'txt_business_type', 'channel', 'record_type_desc', 'record_type_cd', 'producername', 'product_cd', 'product_cd_new', 'renl_cert_no', 'times_renewed_cnt', 'no_of_lives', 'segment_type', 'report_accounting_period', 'veh_model', 'veh_age', 'veh_make', 'veh_body', 'veh_engine_no', 'veh_no', 'binddate', 'no_claim_bonus', 'veh_chassis_no', 'policy_type', 'fuel_type', 'uw_discount', 'uw_loading', 'policy_counter', 'timestamp', 'ncb_percent', 'discount', 'producer_cd', 'major_line_cd', 'minor_line_cd', 'main_lob', 'pol_status_cd', 'productgroup', 'lob', 'minor_line', 'channel_new_2', 'lob_new', 'txt_lob', 'sub_channel_new', 'new_sub_channel', 'group_category', 'producer_type', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured']

In [30]:
sales_all_details_df.persist()
sales_all_details_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

408448

In [31]:
col_str=["plan_type","reference_num", 'accidentaldeathrider']
policy_dh_risk_headers_hlt_df=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_hlt",'public')

col_str=["plan_selected as plan_type","reference_num", "hasbreakinflag as break_in_insurance_flag","breakindays as break_in_insurance_days","exshowroomprice"]
policy_dh_risk_headers_mot_df_plan=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_mot",'public')

col_str=["base_cover as plan_type", "reference_num", 'rider_applicable']
policy_dh_risk_headers_pa_df=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_pa",'public')

col_str=["plan_type", "reference_num", 'dep_date','return_date', 'travel_duration','destination_country','trip', 'zone as lta_zone']
policy_dh_risk_headers_lta_df=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_lta",'public')

col_str=["variant as plan_type", "reference_num", 'coverage_opted', 'age_building', 'personal_accident', 'public_lia']
policy_dh_risk_headers_pne_df=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_pne",'public')

col_str=["trip_type as plan_type", "reference_num",'trip_type','LIMIT', 'purpose_of_trip']
policy_dh_risk_headers_trv_df=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_trv",'public')


policy_dh_risk_union_1=policy_dh_risk_headers_hlt_df.select(['plan_type','reference_num'])\
.union(policy_dh_risk_headers_mot_df_plan.select(['plan_type','reference_num']))

policy_dh_risk_union_2=policy_dh_risk_union_1.select(['plan_type','reference_num'])\
.union(policy_dh_risk_headers_pa_df.select(['plan_type','reference_num']))

policy_dh_risk_union_3=policy_dh_risk_union_2.select(['plan_type','reference_num'])\
.union(policy_dh_risk_headers_lta_df.select(['plan_type','reference_num']))

policy_dh_risk_union_4=policy_dh_risk_union_3.select(['plan_type','reference_num'])\
.union(policy_dh_risk_headers_pne_df.select(['plan_type','reference_num']))

policy_dh_risk_headers_df=policy_dh_risk_union_4.select(['plan_type','reference_num'])\
.union(policy_dh_risk_headers_trv_df.select(['plan_type','reference_num']))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
accidentaldeathrider_hlt_df=policy_dh_risk_headers_hlt_df.select('accidentaldeathrider', 'reference_num')
policy_dh_risk_headers_mot_df=policy_dh_risk_headers_mot_df_plan.select('break_in_insurance_flag', 'break_in_insurance_days', 'exshowroomprice','reference_num' )
rider_pa_df=policy_dh_risk_headers_pa_df.select('rider_applicable', 'reference_num')
pne_df=policy_dh_risk_headers_pne_df.select("reference_num", 'coverage_opted', 'age_building', 'personal_accident', 'public_lia')
trv_df=policy_dh_risk_headers_trv_df.select("reference_num",'trip_type','LIMIT', 'purpose_of_trip')
lta_df=policy_dh_risk_headers_lta_df.select("reference_num", 'dep_date','return_date', 'travel_duration','destination_country','trip', 'lta_zone')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
col_str=["txt_product_code as txt_product_code","txt_uin_number as irda_product_uin",'txt_active_flag']
account_gc_accmst_uin_details_df=load_gpdb_jdbc(col_str,"account_gc_accmst_uin_details",'public').filter((col('txt_active_flag')=='A'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
col_str=["txt_intermediary_code as txt_intermediary_cd","num_reference_number as reference_num"]
underwriting_gc_distribution_channel_tab_df=load_gpdb_jdbc(col_str,"underwriting_gc_distribution_channel_tab",'public')\
.filter((col('reference_num').isNotNull()))
underwriting_gc_distribution_channel_tab_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['txt_intermediary_cd', 'reference_num']

In [35]:
col_str=["trans_id as reference_num","status"]
underwriting_gc_cnfgtr_d_all_transactions_df=load_gpdb_jdbc(col_str,"underwriting_gc_cnfgtr_d_all_transactions",'public')\
.filter((col('trans_id').isNotNull()))
underwriting_gc_cnfgtr_d_all_transactions_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['reference_num', 'status']

In [36]:
col_str=["txt_status","dat_insert_date","dat_modify_date","txt_intermediary_cd"]
reference_gc_genmst_intermediary_df=load_gpdb_jdbc(col_str,"reference_gc_genmst_intermediary",'public').filter((col('txt_intermediary_cd').isNotNull()))
reference_gc_genmst_intermediary_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['txt_status', 'dat_insert_date', 'dat_modify_date', 'txt_intermediary_cd']

In [37]:
col_str=["preexisting_disease as ped_declared_indicator","reference_num","portability_required",'deductible_amount','cumulative_bonus_earned']
policy_dh_risk_details_hlt_df=load_gpdb_jdbc(col_str,"policy_dh_risk_details_hlt",'public').filter((col('reference_num').isNotNull()))
policy_dh_risk_details_hlt_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ped_declared_indicator', 'reference_num', 'portability_required', 'deductible_amount', 'cumulative_bonus_earned']

In [38]:
col_str=["information280","reference_num"]
policy_gc_risk_headers_df=load_gpdb_jdbc(col_str,"policy_gc_risk_headers",'public').filter((col('reference_num').isNotNull()))
policy_gc_risk_headers_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['information280', 'reference_num']

In [39]:
sales_all_details_df_1 = sales_all_details_df.join(policy_dh_risk_headers_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
sales_all_details_df_2 = sales_all_details_df_1.join(policy_dh_risk_headers_mot_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
condition = [sales_all_details_df_2.product_cd == account_gc_accmst_uin_details_df.txt_product_code ]

sales_all_details_df_3 = sales_all_details_df_2.join(account_gc_accmst_uin_details_df,condition,"left").drop("txt_product_code")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
sales_all_details_df_4 =sales_all_details_df_3.join(underwriting_gc_cnfgtr_d_all_transactions_df, "reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
underwriting_gc_distribution_channel_tab_df_1 = underwriting_gc_distribution_channel_tab_df.join(reference_gc_genmst_intermediary_df,"txt_intermediary_cd","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
sales_all_details_df_5 = sales_all_details_df_4.join(underwriting_gc_distribution_channel_tab_df_1,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
sales_all_details_df_6=sales_all_details_df_5.join(accidentaldeathrider_hlt_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
sales_all_details_df_7=sales_all_details_df_6.join(rider_pa_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
sales_all_details_df_8=sales_all_details_df_7.join(pne_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
sales_all_details_df_9=sales_all_details_df_8.join(trv_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
sales_all_details_df_10=sales_all_details_df_9.join(lta_df,"reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
sales_all_details_df_11 = sales_all_details_df_10.join(policy_gc_risk_headers_df, "reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
sales_all_details_df_12 = sales_all_details_df_11.join(policy_dh_risk_details_hlt_df, "reference_num","left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
sales_all_details_df_12.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['reference_num', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system', 'policy_variant', 'mfg_year', 'source_system_customer_id', 'product_name', 'business_type', 'txt_business_type', 'channel', 'record_type_desc', 'record_type_cd', 'producername', 'product_cd', 'product_cd_new', 'renl_cert_no', 'times_renewed_cnt', 'no_of_lives', 'segment_type', 'report_accounting_period', 'veh_model', 'veh_age', 'veh_make', 'veh_body', 'veh_engine_no', 'veh_no', 'binddate', 'no_claim_bonus', 'veh_chassis_no', 'policy_type', 'fuel_type', 'uw_discount', 'uw_loading', 'policy_counter', 'timestamp', 'ncb_percent', 'discount', 'producer_cd', 'major_line_cd', 'minor_line_cd', 'main_lob', 'pol_status_cd', 'productgroup', 'lob', 'minor_line', 'channel_new_2', 'lob_new', 'txt_lob', 'sub_channel_new', 'new_sub_channel', 'group_category', 'producer_type', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured', 'plan_type', 'break_in_insurance_flag', 'break_

In [53]:
sales_all_details_df_q1 = sales_all_details_df_12.selectExpr( 'reference_num', 'policy_number', 'certificate_no', 
'pol_incept_date', 'pol_exp_date', 'source_system','policy_variant', 'mfg_year', 'source_system_customer_id',
'product_name', 'business_type', 'txt_business_type', 'channel', 'record_type_desc', 'record_type_cd', 'producername', 
'product_cd', 'product_cd_new', 'renl_cert_no', 'times_renewed_cnt', 'no_of_lives', 'segment_type', 'report_accounting_period',
'veh_model', 'veh_age', 'veh_make', 'veh_body', 'veh_engine_no', 'veh_no', 'binddate', 'no_claim_bonus', 'veh_chassis_no', 
'binddate as commit_timestamp', 'policy_type', 'fuel_type', 'uw_discount', 'uw_loading', 'policy_counter', 'timestamp', 'ncb_percent',
'discount', 'producer_cd', 'major_line_cd', 'minor_line_cd', 'main_lob', 'pol_status_cd', 'productgroup', 'lob', 
'minor_line', 'channel_new_2', 'lob_new', 'txt_lob', 'sub_channel_new', 'new_sub_channel', 'group_category', 
'producer_type', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured', 'plan_type',
'break_in_insurance_flag', 'break_in_insurance_days', 'exshowroomprice', 'irda_product_uin', 'txt_active_flag', 
'status', 'txt_intermediary_cd', 'txt_status', 'dat_insert_date', 'dat_modify_date', 'accidentaldeathrider',
'rider_applicable', 'coverage_opted', 'age_building', 'personal_accident', 'public_lia', 'trip_type', 'LIMIT as trv_limit', 
'purpose_of_trip', 'dep_date', 'return_date', 'travel_duration', 'destination_country', 'trip', 'lta_zone', 
'information280', 'ped_declared_indicator', 'portability_required', 'deductible_amount', 'cumulative_bonus_earned',
"case when txt_status in ('3','4','5','6') then true else false end as orphan_policy_indicator",
"case when txt_status in ('3','4','5','6') then coalesce(dat_modify_date,dat_insert_date) else null end as orphan_policy_date")\
.withColumn("source_system_name",col('source_system'))\
.withColumn("policy_term",datediff(col("pol_exp_date") ,col("pol_incept_date")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
sales_all_details_df_q1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['reference_num', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system', 'policy_variant', 'mfg_year', 'source_system_customer_id', 'product_name', 'business_type', 'txt_business_type', 'channel', 'record_type_desc', 'record_type_cd', 'producername', 'product_cd', 'product_cd_new', 'renl_cert_no', 'times_renewed_cnt', 'no_of_lives', 'segment_type', 'report_accounting_period', 'veh_model', 'veh_age', 'veh_make', 'veh_body', 'veh_engine_no', 'veh_no', 'binddate', 'no_claim_bonus', 'veh_chassis_no', 'commit_timestamp', 'policy_type', 'fuel_type', 'uw_discount', 'uw_loading', 'policy_counter', 'timestamp', 'ncb_percent', 'discount', 'producer_cd', 'major_line_cd', 'minor_line_cd', 'main_lob', 'pol_status_cd', 'productgroup', 'lob', 'minor_line', 'channel_new_2', 'lob_new', 'txt_lob', 'sub_channel_new', 'new_sub_channel', 'group_category', 'producer_type', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured', 'plan_type', 'break_in_insu

In [55]:
# sales_all_details_df_q1 = sales_all_details_df_12.selectExpr('reference_num', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system', 'partner_appl_no', 'branch', 'report_accounting_period', 'receipt_no', 'discount', 'endorsement_no', 'leader_name', 'group_category', 'main_lob', 'major_line_cd', 'mfg_year', 'minor_line_cd', 'veh_make', 'veh_model', 'ri_inward_flag', 'zone', 'product_cd_new', 'booking_date', 'channel', 'class_peril_cd', 'policy_term', 'policy_type', 'policy_variant', 'coinsurance_flag', 'application_no', 'renl_cert_no', 'record_type_desc', 'record_type_cd', 'producer_cd', 'producername', 'product_name', 'CommissionPercent', 'main_receipt_no', 'share', 'business_type', 'segment_type', 'sum_insured', 'rural_fg', 'branch_off_cd', 'producer_type', 'pin', 'prdr_branch_sub_cd', 'times_renewed_cnt', 'txt_business_type', 'covernotenumber', 'watts_policy_no', 'coinsurance_cd', 'timestamp', 'veh_engine_no', 'veh_no', 'veh_body', 'source_system_customer_id', 'customer_id', 'clientname', 'client_city_name', 'endorsement_eff_date', 'prem_eff_date', 'sme_flag', 'fac_prem', 'ceded_vqst_prem', 'ceded_surplus_treaty_prem', 'obligatory_prem', 'obligatory_comm', 'pol_office_cd', 'producer_name', 'ncb_percent', 'pol_status_cd', 'product_cd', 'veh_age', 'txt_chasis_no', 'uw_discount', 'uw_loading', 'fuel_type', 'display_product_cd', 'policy_counter', 'no_of_lives', 'row_num', 'product_eff_date', 'BindDate as commit_timestamp', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured', 'lob', 'Channel_New_2', 'plan_type', 'break_in_insurance_flag', 'break_in_insurance_days', 'exshowroomprice', 'irda_product_uin', 'txt_active_flag', 'status', 'txt_intermediary_cd', 'txt_status', 'dat_insert_date', 'dat_modify_date', 'accidentaldeathrider', 'rider_applicable', 'coverage_opted', 'age_building', 'personal_accident', 'public_lia', 'trip_type', 'LIMIT', 'purpose_of_trip', 'dep_date', 'return_date', 'travel_duration', 'destination_country', 'trip', 'lta_zone', 'information280', 'ped_declared_indicator', 'portability_required', 'deductible_amount', 'cumulative_bonus_earned',
#  "case when txt_status in ('3','4','5','6') then true else false end as orphan_policy_indicator",
#         "case when txt_status in ('3','4','5','6') then coalesce(dat_modify_date,dat_insert_date) else null end as orphan_policy_date")\
#     .withColumn("source_system_name",col('source_system'))\
#     .withColumn("policy_term",datediff(col("pol_exp_date") ,col("pol_incept_date")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
sales_all_details_df_q1=sales_all_details_df_q1.filter((col('policy_number').isNotNull()))
sales_all_details_df_q1 = sales_all_details_df_q1.drop_duplicates()

w = Window.partitionBy(['policy_number','certificate_no','pol_incept_date','pol_exp_date','source_system'])\
    .orderBy([F.col('commit_timestamp').desc(),F.col('reference_num').desc()])
sales_all_details_df_q1 = sales_all_details_df_q1.withColumn('row',F.row_number().over(w))
sales_all_details_df_q1 = sales_all_details_df_q1.filter(F.col('row')==1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
col_str=["reference_num","LDDESC"]
reference_gc_cnfgtr_policy_ld_dtls_df=load_gpdb_jdbc(col_str,"reference_gc_cnfgtr_policy_ld_dtls",'public')\
.filter(col("LDDESC").isin('No Claim Bonus'))
reference_gc_cnfgtr_policy_ld_dtls_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['reference_num', 'LDDESC']

In [58]:
sales_all_details_df_q2 = sales_all_details_df.join(reference_gc_cnfgtr_policy_ld_dtls_df,\
      "reference_num","inner").withColumn("ncb_flag",lit(True)).filter((col('policy_number').isNotNull()))\
.select(['reference_num','ncb_flag'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
df6 = sales_all_details_df_q1.join(sales_all_details_df_q2,'reference_num',"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
policy_dh_risk_details_hlt_df_2 = policy_dh_risk_details_hlt_df.filter(col("portability_required")=="True")\
    .selectExpr("portability_required","reference_num")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
sales_all_details_df_q1_q4 = sales_all_details_df.join(policy_dh_risk_details_hlt_df_2, "reference_num","inner")\
    .selectExpr("reference_num ").withColumn("portability_indicator",lit(True))
sales_all_details_df_q1_q4.columns
sales_all_details_df_q1_q4 = sales_all_details_df_q1_q4.groupBy('reference_num').agg(F.max(F.col('portability_indicator')).alias("portability_indicator"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df8 = df6.join(sales_all_details_df_q1_q4,'reference_num',"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
grp=['policy_number','certificate_no']
windowSpec  = Window.partitionBy(grp).orderBy(col("pol_incept_date").desc())

sales_all_details_df_q5 = sales_all_details_df1.withColumn("new_pol_incept_date",max(col("pol_incept_date")).over(windowSpec))\
.withColumn("dense_rank",dense_rank().over(windowSpec))\
.selectExpr("pol_exp_date as previous_pol_exp_date","new_pol_incept_date","dense_rank",'pol_incept_date as previous_pol_incept_date',"reference_num")


sales_all_details_df_q5 = sales_all_details_df.join(sales_all_details_df_q5,"reference_num","left")\
    .selectExpr("policy_number","reference_num ","previous_pol_exp_date","new_pol_incept_date","dense_rank")\
    .filter(col("dense_rank") == '2' )\
    .withColumn("renewed_in_grace_period",when(date_add(col("previous_pol_exp_date"),1) < col("new_pol_incept_date"),lit(True))\
               .otherwise(lit(False)))\
    .selectExpr("reference_num","renewed_in_grace_period")
sales_all_details_df_q5 = sales_all_details_df_q5.groupBy('reference_num').agg(F.max(F.col('renewed_in_grace_period')).alias("renewed_in_grace_period"))    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
final_df = df8.join(sales_all_details_df_q5,'reference_num',"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
final_df=final_df.withColumn("renewed_in_grace_period",col("renewed_in_grace_period").cast(BooleanType()))


w = Window.partitionBy(['policy_number','certificate_no','pol_incept_date', 'pol_exp_date','source_system_name']).orderBy([F.col('commit_timestamp').desc(),F.col('reference_num').desc()])
df_main = final_df.withColumn('row',F.row_number().over(w))
df_main = df_main.filter(F.col('row')==1).\
filter((col('policy_number').isNotNull())).filter((col('source_system_customer_id').isNotNull())).drop(df_main.row)

    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
final_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['reference_num', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system', 'policy_variant', 'mfg_year', 'source_system_customer_id', 'product_name', 'business_type', 'txt_business_type', 'channel', 'record_type_desc', 'record_type_cd', 'producername', 'product_cd', 'product_cd_new', 'renl_cert_no', 'times_renewed_cnt', 'no_of_lives', 'segment_type', 'report_accounting_period', 'veh_model', 'veh_age', 'veh_make', 'veh_body', 'veh_engine_no', 'veh_no', 'binddate', 'no_claim_bonus', 'veh_chassis_no', 'commit_timestamp', 'policy_type', 'fuel_type', 'uw_discount', 'uw_loading', 'policy_counter', 'timestamp', 'ncb_percent', 'discount', 'producer_cd', 'major_line_cd', 'minor_line_cd', 'main_lob', 'pol_status_cd', 'productgroup', 'lob', 'minor_line', 'channel_new_2', 'lob_new', 'txt_lob', 'sub_channel_new', 'new_sub_channel', 'group_category', 'producer_type', 'min_pol_incept_date', 'premiumamount', 'total_premium', 'base_sum_insured', 'plan_type', 'break_in_insu

In [67]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_main.count()
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

408448

In [68]:
df_main=df_main.withColumnRenamed('base_sum_insured','sum_insured')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
output_index = "policy_retail_staging"  #policy_retail_staging_211123
schema = "customermart"

import sys
try:

    df_main.write.format("greenplum").option("truncate", "true")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [72]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count,least
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-03-08 10:02:12.143816'

In [73]:
table_name='policy_retail'
source = "policy_retail"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
import sys
try:
    update_policy(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Inside try segment
Inside except segment
Executed update query
Executed insert query
success

In [ ]:
if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')

In [ ]:
df_main.unpersist()
sales_all_details_df.unpersist()

In [ ]:
conn_prod.close()

In [ ]:
import io
import pandas as pd
import xlsxwriter
import sys
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import pandas as pd
from sqlalchemy import create_engine
from smtplib import SMTP

In [ ]:
if x!='success':
    status='FAILED'
else:
    status="SUCCESS"

In [ ]:
subject=f"Automated Mail-POLICY Job Status : {status}"

In [ ]:
if x=='success':
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        Policy job run is Successful
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())
else:
      
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com']
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        Policy job run FAILED
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())

In [75]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…